In [7]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import torch.utils.data as Data

### 一、数据读取csv

In [2]:
# 读取并显示数据的前几行
spam=pd.read_csv('./spambase.csv')
spam.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,label
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


#### 获取自变量、因变量，并划分训练集测试集：

In [4]:
X=spam.iloc[:,0:57].values
y=spam.label.values     # ndarray数组类型
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=123)
# 随机划分的

#### 也可以加上标准化的步骤，这里重点在于数据格式的转换，故不在加上这一步

### 二、将数据转换为张量

In [6]:
X_train_t=torch.from_numpy(X_train.astype(np.float32))
y_train_t=torch.from_numpy(y_train.astype(np.int64))    # 注意！！！这里因变量y是分类数据，所以是np.int64类型，若是回归问题，则与x同。
X_test_t=torch.from_numpy(X_test.astype(np.float32))
y_test_t=torch.from_numpy(y_test.astype(np.int64))

### 三、使用TensorDataset将数据X和y整合到一起

In [8]:
train_data=Data.TensorDataset(X_train_t,y_train_t)
test_data=Data.TensorDataset(X_test_t,y_test_t)

### 四、定义一个数据加载器，对训练数据集进行批量处理

In [10]:
train_loader=Data.DataLoader(
    dataset=train_data,    # 使用的数据集
    batch_size=64, # 批处理样本大小
    shuffle=True,    # 每次迭代前打乱数据
    num_workers=1
)

test_loader=Data.DataLoader(
    dataset=test_data,    # 使用的数据集
    batch_size=64, # 批处理样本大小
    shuffle=True,    # 每次迭代前打乱数据
    num_workers=1
)

### 至此，已经完成了csv到train_loader的转换，下面验证一下效果：

In [11]:
# 获取要给batch的数据：
for step,(b_x,b_y) in enumerate(train_loader):
    if step>0:
        break
print(type(train_loader))
print(type(b_x))
print(b_x.shape)

<class 'torch.utils.data.dataloader.DataLoader'>
<class 'torch.Tensor'>
torch.Size([64, 57])


### 发现，b_x的shape为64*57，其中57为特征个数，而64则为一个batch中的样本个数。